## lib load

In [1]:
%matplotlib widget

In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches

import numpy as np
from scipy import optimize

from ACqubit import *
from ACstate import *
from wavefunction1d import *

#from tqdm import tqdm, tqdm_notebook
from tqdm.autonotebook import tqdm


import qcodes as qc
from qcodes.dataset.database import initialise_database
from qcodes.dataset.plotting import plot_by_id, get_data_by_id


pi = np.pi

import matplotlib.transforms as mtransforms

D:\anaconda\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
D:\anaconda\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'


# Double gating

In [24]:
pcolor


<function __main__.pcolor(x, y, z, size=(5, 4))>

In [49]:
file = 'expdata\\2d gating.txt'

data = np.genfromtxt(file)

x = data[0 ,1:]
y = data[1:,0]

X, Y = np.meshgrid(x, y)

Z = data[1:, 1:]

fig, ax = plt.subplots()
qm2 = ax.pcolormesh(X, Y, Z)#, transform = mtransforms.Affine2D().rotate_deg(3))

trans = mtransforms.Affine2D().from_values(1, 0.9, 2,1, 0, 0)
qm2.set_transform(trans + qm2.get_transform())

ax.set_xlim(-6,6)
ax.set_ylim(-4,4)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


FigureCanvasNbAgg()

(-4, 4)

In [29]:
axs.get_ 

# 2-bf

In [6]:
def find_nearest(array, value): # https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
def pcolor(x,y,z, size=(5,4)):
    f = plt.figure(figsize=size)
    ax = f.add_subplot(111)
    x1, y1 = np.meshgrid(x,y)
    #mesh = ax.pcolormesh(y1, x1 ,z, norm=colors.LogNorm())
    im = mesh = ax.pcolormesh(y1,x1,z)
    f.colorbar(im, ax=ax)
    return ax, mesh

################################################################################################
## Open a txt file, turn it into a dictionary. The dictionary has n1, n2, and E01 ##
################################################################################################
def open_and_extract_txt_E01(txtfile='', xname='', yname='', zname='', size=None):
    file = open(txtfile, "r")
    a = file.read().split('\n')
    del(a[-1])
    print('# of entries in .txt: ' + str(len(a)))
    file.close()
    x = []
    y = []
    e01 = []
    for i in range(len(a)):
        if i%3==0:
            x.append(float(a[i]))
        if i%3==1:
            y.append(float(a[i]))
    x = np.unique(np.asarray(x))
    y = np.unique(np.asarray(y))
    for i in range(len(a)):
        if i%3==2:
            e01.append(float(a[i]))
    e01 = np.reshape(e01,(size,size))
    print('Size of ' + str(xname) + ': ' + str(np.shape(x)))
    print('Size of ' + str(yname) + ': ' + str(np.shape(y)))
    print('Size of ' + str(zname) + ': ' + str(np.shape(e01)))
    return {'x': x, 'y': y, 'z': e01}



################################################################################################
## Open a txt file, turn it into a dictionary. The dictionary has n1, n2, E0, and E1 ###########
################################################################################################
def open_and_extract_txt_E0E1(txtfile='', xname='', yname='', z1name='', z2name='', size=None):
    file = open(txtfile, "r")
    a = file.read().split('\n')
    del(a[-1])
    print('# of entries in .txt: ' + str(len(a)))
    file.close()
    x = []
    y = []
    e0 = []
    e1 = []
    for i in range(len(a)):
        if i%4==0:
            x.append(float(a[i]))
        if i%4==1:
            y.append(float(a[i]))
    x = np.unique(np.asarray(x))
    y = np.unique(np.asarray(y))
    for i in range(len(a)):
        if i%4==2:
            e0.append(float(a[i]))
    for i in range(len(a)):
        if i%4==3:
            e1.append(float(a[i]))
    e0 = np.reshape(e0,(size,size))
    e1 = np.reshape(e1,(size,size))
    print('Size of ' + str(xname) + ': ' + str(np.shape(x)))
    print('Size of ' + str(yname) + ': ' + str(np.shape(y)))
    print('Size of ' + str(z1name) + ': ' + str(np.shape(e0)))
    print('Size of ' + str(z2name) + ': ' + str(np.shape(e1)))

    return {'x': x, 'y': y, 'z1': e0, 'z2': e1}

In [7]:
## Define the dictionary with n1, n2, E0, and E1 
A = open_and_extract_txt_E0E1("BBFL_n1n2E0E1_0150_eigen_pi_cluster.txt",xname='n1', yname='n2', z1name='e0',z2name='e1', size=50)

## Make a pretty plot
axs, meshs = pcolor(A['x'],A['y'], A['z1'])
axs.set_xlabel('n1')
axs.set_ylabel('n2')
#axs.set_yticks([0., .5*np.pi, np.pi, 1.5*np.pi])
#axs.set_yticklabels(["$0$", r"$\frac{\pi}{2}$",
#                 r"$\pi$", r"$\frac{3\pi}{2}$"])
axs.set_title('BBFL E0  phi=π')

# of entries in .txt: 10000
Size of n1: (50,)
Size of n2: (50,)
Size of e0: (50, 50)
Size of e1: (50, 50)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'BBFL E0  phi=π')

In [8]:
x = A['x']
y = A['y']
X, Y = np.meshgrid(x, y)

## Make a prettier plot
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(X, Y, A['z2']-A['z1'], rstride=1, cstride=1, cmap=cm.viridis)
#ax.plot_surface(X, Y, A['z2'], rstride=1, cstride=1, cmap=cm.viridis)
# ax.set_xlabel(r'$n_g^{(1)}$', fontsize = 18)
# ax.set_ylabel(r'$n_g^{(2)}$', fontsize = 18)
# ax.set_title('BBFL - E01')

ax.tick_params(axis="x", labelsize=15)
ax.tick_params(axis="y", labelsize=15)
ax.tick_params(axis="z", labelsize=15)

ax.set_xticks([0., 0.25, 0.5,  0.75, 1])
ax.set_xticklabels([0., '', 0.5,  '', 1])

ax.set_yticks([0., 0.25, 0.5,  0.75, 1])
ax.set_yticklabels([0., '', 0.5,  '', 1])

ax.set_zticks([])

# ax.set_zlabel(r'$E_{01}$', fontsize = 18)
plt.show()

FigureCanvasNbAgg()

AttributeError: 'QuadMesh' object has no attribute 'do_3d_projection'

In [16]:
spec = A['z2']-A['z1']

spec_2bf = [spec[i][i] for i in range(len(x))]


In [23]:
J1 = 27*1
J2 = 1.0*J1

ACQ_ = ACQubit (E_CL = 7, 
              E_L = 0.9, 
              E_J = (J1+J2)/2, 
              E_C = 10,
              dE_J = 6 )

In [21]:
# EL = 0.00001
EL = 0.0001

Qideal =  ACQubit (E_CL = 7, 
              E_L = EL, 
              E_J = 70, 
              E_C = 35,
              dE_J = 7 )

In [34]:
# Q = ACQB15
# Q = ACQ_
Q = Qideal

N_ng = 101


ng_list = np.linspace(0.0, 1, N_ng)

 
E_n = Q.iterate_ng( ng_list, 2*pi*0.25, 'get_E') 

spec_1bf = E_n[:,1] - E_n[:,0]

In [35]:
fig, ax = plt.subplots()

# ax.plot(x, spec_2bf, label = '2-island bf')
ax.plot(ng_list, spec_1bf, label = '1-island bf')

ax.legend()

ax.set_xlabel(r'$n_{g1} = n_{g2}$')
ax.set_ylabel(r'$E_{01}$, GHz')

FigureCanvasNbAgg()

Text(0, 0.5, '$E_{01}$, GHz')